In [1]:
import cv2
import numpy as np

In [12]:
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

input_img = cv2.imread('group_img_mask.jpg')
gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
faces = face_detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors =5)

for (x, y, w, h) in faces:
    cv2.rectangle(input_img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
cv2.imshow('image', input_img)
cv2.waitKey(0)
cv2.destroyAllWindows()